<a href="https://colab.research.google.com/github/Jontpan/master-thesis/blob/master/QCBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install wandb

     |████████████████████████████████| 2.1MB 9.9MB/s 
     |████████████████████████████████| 870kB 36.7MB/s 
     |████████████████████████████████| 3.3MB 34.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=539596a3c372cfc4253a65490c3f6f3004c2be894d7ef5b3e9bebe72ec4eceec
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
     |████████████████████████████████| 2.1MB 11.3MB/s 
     |████████████████████████████████| 163kB 38.2MB/s 
     |████████████████████████████████| 133kB 38.9MB/s 
     |████████████████████████████████| 102kB 9.5MB/s 
     |████████████████████████████████| 71kB 7.1MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=7bf76bb7908939dc0740954d4a77ead27733770cad0640beacdea3026a0b41b3
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68

In [ ]:
import os
import re
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wandb

wandb.login()

%matplotlib inline
%load_ext tensorboard

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
%env WANDB_LOG_MODEL=true

env: WANDB_LOG_MODEL=true


In [ ]:
train_data = pd.read_csv('drive/MyDrive/master-thesis/data/swe_qaqc_train.csv')
test_data = pd.read_csv('drive/MyDrive/master-thesis/data/swe_qaqc_test.csv')


train_data.sample(5)

,text,verbose label,course label,fine label
1238,Hur mycket energi släpps ut när syre och väte ...,NUM:count,5,38
5400,Vem vann det första allmänna valet för preside...,HUM:ind,1,30
1410,"Vad var rätt Laugh-In svar till: ""Say goodnigh...",DESC:desc,2,3
3989,Vem gjorde det första flygplanet som kunde flyga?,HUM:gr,1,29
4846,Vem äger CNN?,HUM:ind,1,30


In [ ]:
from sklearn.model_selection import train_test_split

X_train = train_data['text'].tolist()
y_train = train_data['fine label'].tolist()
X_test = test_data['text'].to_list()
y_test = test_data['fine label'].to_list()


In [ ]:
from transformers import AutoTokenizer
tok = AutoTokenizer.from_pretrained('KB/bert-base-swedish-cased')

# Concatenate train data and test data
all_texts = np.concatenate([X_train, X_test])

# Encode our concatenated data
encoded_texts = [tok.encode(sent, add_special_tokens=True) for sent in all_texts]

# Find the maximum length
max_len = max([len(sent) for sent in encoded_texts])
print('Max length: ', max_len)


Max length:  43


In [ ]:
train_encodings = tok(list(X_train), truncation=True, padding=True, max_length=max_len)
test_encodings = tok(list(X_test), truncation=True, padding=True, max_length=max_len)

In [ ]:
for idx, ids in enumerate(train_encodings['input_ids']):
    if idx == 3240:
        print(ids)
        print(tok.decode(ids))
        print(tok.convert_ids_to_tokens(ids))
        break


[2, 3755, 3618, 358, 97, 1424, 37769, 65, 23521, 6, 302, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[CLS] Vem anses vara den största tillverkaren av fioler? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
['[CLS]', 'Vem', 'anses', 'vara', 'den', 'största', 'tillverkaren', 'av', 'fiol', '##er', '?', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [ ]:
class QAQCDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = QAQCDataset(train_encodings, y_train)
test_dataset = QAQCDataset(test_encodings, y_test)

In [ ]:
%%time
from transformers import AutoModel
import torch.nn as nn
from transformers.modeling_outputs import MultipleChoiceModelOutput

class SwEAT_BERT(nn.Module):

    def __init__(self, num_labels):
        super(SwEAT_BERT, self).__init__()
        self.num_labels = num_labels
        self.bert = AutoModel.from_pretrained('KB/bert-base-swedish-cased')
        self.dropout = nn.Dropout(self.bert.config.hidden_dropout_prob)
        self.classifier = nn.Linear(self.bert.config.hidden_size, self.num_labels)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        ):

        return_dict = return_dict if return_dict is not None else self.bert.config.use_return_dict
        num_choices = self.num_labels

        input_ids = input_ids.view(-1, input_ids.size(-1)) if input_ids is not None else None
        attention_mask = attention_mask.view(-1, attention_mask.size(-1)) if attention_mask is not None else None
        token_type_ids = token_type_ids.view(-1, token_type_ids.size(-1)) if token_type_ids is not None else None
        position_ids = position_ids.view(-1, position_ids.size(-1)) if position_ids is not None else None
        inputs_embeds = (
            inputs_embeds.view(-1, inputs_embeds.size(-2), inputs_embeds.size(-1))
            if inputs_embeds is not None
            else None
        )

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        reshaped_logits = logits.view(-1, num_choices)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(reshaped_logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return MultipleChoiceModelOutput(
            loss=loss,
            logits=reshaped_logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

CPU times: user 126 ms, sys: 5.63 ms, total: 132 ms
Wall time: 152 ms


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    prec, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    cm = confusion_matrix(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'recall': recall,
        'precision': prec,
    }

In [ ]:
def model_init(num_labels):
    return SwEAT_BERT(num_labels=num_labels)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = './results',
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=0,              # number of warmup steps for learning rate scheduler
    learning_rate=5e-5,
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    run_name='new run'
)

model = model_init(50)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,             # evaluation dataset
    compute_metrics=compute_metrics      # evaluation metrics
)

In [ ]:
trainer.train()

wandb: Currently logged in as: jontpan (use `wandb login --relogin` to force relogin)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:760: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Step,Training Loss
100,2.270100
200,1.130100
300,0.901800
400,0.733800
500,0.489200
600,0.446400
700,0.458000
800,0.329500
900,0.231400
1000,0.234700


TrainOutput(global_step=1492, training_loss=0.5289649733269822, metrics={'train_runtime': 571.7545, 'train_samples_per_second': 2.61, 'total_flos': 0.0, 'epoch': 4.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 500037120, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 18194432, 'train_mem_gpu_alloc_delta': 1507572736, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 0})

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'epoch': 4.0,
 'eval_accuracy': 0.91,
 'eval_f1': 0.9073795482295481,
 'eval_loss': 0.4934951663017273,
 'eval_mem_cpu_alloc_delta': 331776,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 46306816,
 'eval_precision': 0.9130628791992321,
 'eval_recall': 0.91,
 'eval_runtime': 1.4378,
 'eval_samples_per_second': 347.755}

In [ ]:
torch.save(model.state_dict(), 'drive/MyDrive/master-thesis/models/SWEQCBERT_50.bin')